1. Finding relationship from gcreplay data

In [5]:
import pandas as pd, csv
df_sample_ids = pd.read_csv('gctree-node-data-small.csv', skip_blank_lines=True, usecols=['sampled_cell_ids', 'name', 'abundance', 'descendant_abundance', 'delta_bind_CGG_tdms_linear_model_pred', 'delta_bind_CGG_tdms_model_pred'],verbose=True)
print(df_sample_ids)

Tokenization took: 0.74 ms
Type conversion took: 0.66 ms
Parser memory cleanup took: 0.01 ms
     name  abundance                              sampled_cell_ids  \
0   naive          0                                           NaN   
1       2          0                                           NaN   
2       8          0                                           NaN   
3   seq19          1                                220114P03G02HK   
4       1          0                                           NaN   
5   seq31          1                                220114P03G12HK   
6       7          0                                           NaN   
7   seq35          1                                220114P03H05HK   
8   seq10          1                                220114P03E10HK   
9    seq2          1                                220114P03D04HK   
10  seq33          1                                220114P03H03HK   
11      6          0                                           NaN 

2. Getting evaluation from torchdms and predicting theta to compare

In [11]:
def get_frac_antigen_bound(delta_log10_KD: float, log10_naive_KD: float, concentration_antigen: float):
    log10_KD  = delta_log10_KD + log10_naive_KD
    KD = 10**log10_KD
    # Hill equation with n = 1:
    theta = concentration_antigen/(KD + concentration_antigen)
    return theta

log10_naive_KD = -10.43
concentration_antigen = 10**(-8)

sampled_cell_ids = []
fitnesses = []
antigen_bound_fracs = []

for idx, row in df_sample_ids.iterrows():
    if int(row.abundance) > 0:
        sampled_cell_ids.append(row.sampled_cell_ids)
        fitnesses.append(float(row.descendant_abundance)/float(row.abundance))
        antigen_bound_fracs.append(get_frac_antigen_bound(row.delta_bind_CGG_tdms_linear_model_pred, log10_naive_KD, concentration_antigen))
    
antigen_df = pd.DataFrame({"sampled_cell_id": sampled_cell_ids, "antigen_bound_fracs": antigen_bound_fracs, "fitnesses": fitnesses})
print(antigen_df)

                                 sampled_cell_id  antigen_bound_fracs  \
0                                 220114P03G02HK             1.000000   
1                                 220114P03G12HK             0.541644   
2                                 220114P03H05HK             0.999998   
3                                 220114P03E10HK             1.000000   
4                                 220114P03D04HK             0.990697   
5                                 220114P03H03HK             0.541644   
6                                 220114P03D12HK             0.976553   
7   220114P03D10HK,220114P03F10HK,220114P03H04HK             0.946610   
8                                 220114P03B04HK             0.962320   
9                  220114P03D09HK,220114P03C12HK             0.988622   
10                                220114P03E09HK             0.956749   
11                                220114P03A10HK             0.967627   
12                                220114P03G05HK   